In [1]:
# --- Import required packages
import pathlib
import pandas as pd
import numpy as np
from itertools import combinations
import os
from tqdm import tqdm
import requests
from datetime import datetime
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
import platform

# Import helper functions
from logit_demand_functions import *

# Warnings and display
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)

In [2]:
# --- Set paths ----------------
if platform.platform()[0:5] == 'macOS':
    cd = pathlib.Path().resolve().parent.parent
    str_project = cd / "Documents" 
    data_path = cd / "Documents" / "tobin_working_data"
    rlpolk_data_path = data_path / "rlpolk_data"
    vin_matching_path =  data_path / "vin_matching"
    data_one_vins_path = data_path / "vin_decoder"
else:
    data_path = pathlib.Path().resolve().parent.parent / "rn_home" / "data"
    rlpolk_data_path = data_path / "rlpolk_data"
    vin_matching_pathpackages= data_path / "vin_matching"

In [3]:
# --- Import RLPolk Data (partially matched) ------
rlp_raw = pd.read_csv(rlpolk_data_path / "rlpolk_data_matched.csv", index_col = [0])
evs_only = rlp_raw[rlp_raw["Fuel Type - Primary"] == "Electric"].reset_index(drop = True)
ev_vins = evs_only["vin_corrected"].unique().tolist()

# Make the rlp raw column names lower case
rlp_raw.columns = [x.lower() for x in rlp_raw.columns]

# Drop duplicated rlp columns
rlp_raw = rlp_raw.loc[:,~rlp_raw.columns.duplicated()]


In [4]:
# --- Import Experian Data
# exp_data = pd.read_csv(data_path / "intermediate" / "US_VIN_data_common.csv")
# exp_ct = exp_data[exp_data["state"]=="CONNECTICUT"]
# exp_ct_ev = exp_ct[exp_ct["fueltype"]=="L"]

# Download VIN characteristics from NHTSA

In [5]:
if (False):
    # Download some characteristics and show what we get - mostly NA
    if (True):
        ev_chars = pd.DataFrame([])

        for vin in tqdm(ev_vins):
            # vin = row["vin_corrected"]
            try:
                chars = fetch_vin_data(vin)
                # print(len(chars.columns))
            except:
                pass
            try:   
                ev_chars = pd.concat([ev_chars, chars], axis = 0).reset_index(drop = True)
            except:
                pass


        ev_chars.to_csv(rlpolk_data_path / "rlp_ev_characteristics_012624.csv")

    # Check EV sales per model year
    rlp_evs_summary = evs_only[["year", "VEH_COUNT"]].groupby("year").sum().reset_index()
    # exp_evs_summary = exp_ct_ev[["year", "agg_count"]].groupby("year").sum().reset_index()
    print(rlp_evs_summary.head(10))
    # print(exp_evs_summary.head(10))
    print(rlp_evs_summary.iloc[0:6, 0].sum())

# Attempt match with DataOne Matcher

In [6]:
# Import RLP data
rlp_raw = pd.read_csv(rlpolk_data_path / "rlpolk_data_matched.csv", index_col = [0])

# Import DataOne Data
d1_vins = pd.read_csv(data_one_vins_path / "VIN_REFERENCE.csv")
d1_vins["d1_merge"] = 1
d1_prices = pd.read_csv(data_one_vins_path / "VEH_PRICE.csv")

# Merge d1_prices and d1_vins on vehicle id
d1_vins = d1_vins.merge(d1_prices, on = "vehicle_id", how = "left")

# Make all the columns in RLP data lower case and drop the duplicate columns
rlp_raw.columns = [x.lower() for x in rlp_raw.columns]
rlp_raw = rlp_raw.loc[:,~rlp_raw.columns.duplicated()]

# Create a column in rlp that is 1 if the transaction_price is missing, and 0 otherwise
rlp_raw["rlp_missing_price"] = rlp_raw["transaction_price"].isna().astype(int)

# Create a column in rlp that is 1 if the transaction price is below 10000 and 0 otherwise
rlp_raw["rlp_low_price"] = (rlp_raw["transaction_price"] < 10000).astype(int)

In [7]:
# Confirm lengths align
print(f"All RLP VINS are {rlp_raw.vin_prefix.str.len().unique().tolist()[0]} long")
print(f"All DataOne VINS are {d1_vins.vin_pattern.str.len().unique().tolist()[0]} long")

All RLP VINS are 10 long
All DataOne VINS are 10 long


In [8]:
# Provide breakdown of how many in RLP have missing price
print(rlp_raw["rlp_missing_price"].value_counts())

# Provide breakdown of how many in RLP have low price
print(rlp_raw["rlp_low_price"].value_counts())

rlp_missing_price
0    672478
1    251999
Name: count, dtype: int64
rlp_low_price
0    902596
1     21881
Name: count, dtype: int64


In [9]:
# Drop those in RLP with a missing or low price
rlp_raw_filtered = rlp_raw[(rlp_raw["rlp_missing_price"] == 0) & (rlp_raw["rlp_low_price"] == 0)].reset_index(drop = True)

In [10]:
# Prepare for match
rlp_raw_filtered["vin_d1_match"] = rlp_raw_filtered.apply(lambda x: x["vin_prefix"][0:8] + x["vin_prefix"][9] if x["vin_prefix"] is not np.nan else np.nan, axis = 1)
d1_vins["vin_rlp_match"] = d1_vins["vin_pattern"].str[0:9]

In [11]:
# Do a fuzzy match first on vins and then on msrp and transaction price
def fuzzy_match(vin, price, df_match_from, keycol, pricecol, other_cols):
    try:
        possible_matches = df_match_from.loc[df_match_from[keycol]==vin, :]
        # display(possible_matches)
        # print((possible_matches[pricecol]-price).abs().argsort()[:1].index.tolist()[0])
        closest_row = df_match_from.iloc[[(possible_matches[pricecol]-price).abs().argsort()[:1].index.tolist()[0]], :]
        return closest_row
    except:
        return None

In [12]:
if (False):
    # closest_matches_temp = pd.DataFrame([])
    closest_matches = [ ]

    for index, row in tqdm(rlp_raw_filtered.iterrows(), total = len(rlp_raw_filtered)):
        try:
            match = fuzzy_match(row.vin_d1_match, row.transaction_price, d1_vins, "vin_rlp_match", "msrp", ["vin_rlp_match", "msrp"])
            match["new_index"] = index
            closest_matches.append(match) 
            #closest_matches_temp = pd.concat([closest_matches_temp, match], axis = 0)
            # if (index % 1000) == 0:
            #     closest_matches = pd.concat([closest_matches, closest_matches_temp], axis = 0)
            #     closest_matches_temp = pd.DataFrame([])
        except:
            pass

    closest_matches_df = pd.concat(closest_matches)
    closest_matches_df.to_csv(rlpolk_data_path / "rlp_d1_closest_matches_2.csv")

In [71]:
# Reload closest matches from where we saved it
closest_matches = pd.read_csv(rlpolk_data_path / "rlp_d1_closest_matches_2.csv", index_col = [0])


In [72]:
# Add the suffix _D1 to all columns in closest_matches except new_index and vin_rlp_match
closest_matches.columns = [x + "_D1" for x in closest_matches.columns if not x == "new_index"]+ ["new_index"]

# Merge closest matches and rlp_filtered_raw on index and new_index, and _d1 prefix to all columns from closest_matches
# Keep the index from rlp_filtered_raw
rlp_raw_filtered_formerge = rlp_raw_filtered.reset_index()
rlp_raw_filtered_formerge = rlp_raw_filtered_formerge.rename(columns = {"index": "old_index"})
rlp_d1_merge = rlp_raw_filtered_formerge.merge(closest_matches, left_on = "old_index", right_on = "new_index", how = "left")

# Drop those entries with no match
rlp_d1_merge = rlp_d1_merge[rlp_d1_merge.new_index.notna()]

# View columns
print(rlp_d1_merge.columns)


Index(['old_index', 'year_month', 'make', 'model', 'model_year',
       'transaction_price', 'zip_code', 'vin_prefix', 'county_name',
       'state_abbrv', 'veh_count', 'vin_corrected', 'manufacturer name',
       'model year', 'fuel type - primary', 'electrification level', 'year',
       'month', 'rlp_missing_price', 'rlp_low_price', 'vin_d1_match',
       'vin_id_D1', 'vehicle_id_D1', 'vin_pattern_D1', 'year_D1', 'make_D1',
       'model_D1', 'trim_D1', 'style_D1', 'mfr_model_num_D1',
       'mfr_package_code_D1', 'doors_D1', 'drive_type_D1', 'vehicle_type_D1',
       'rear_axle_D1', 'body_type_D1', 'body_subtype_D1', 'bed_length_D1',
       'engine_id_D1', 'engine_name_D1', 'engine_size_D1', 'engine_block_D1',
       'engine_cylinders_D1', 'engine_valves_D1', 'engine_induction_D1',
       'engine_aspiration_D1', 'engine_cam_type_D1', 'fuel_type_D1',
       'trans_id_D1', 'trans_name_D1', 'trans_type_D1', 'trans_speeds_D1',
       'wheelbase_D1', 'gross_vehicle_weight_range_D1', 're

In [69]:
rlp_d1_merge[["old_index", "new_index"]]

,old_index,new_index
0,0,0.0
1,1,1.0
2,2,2.0
3,3,3.0
4,4,4.0
...,...,...
650592,650592,650592.0
650593,650593,650593.0
650594,650594,650594.0
650595,650595,650595.0


In [70]:
# View the make and model columns, for the top 10 entries
display(rlp_d1_merge[["new_index", "make", "model", "make_D1", "model_D1"]].head(10))

# Print a divider
print("=============================================================")

# View the same for the bottom 10 entries
display(rlp_d1_merge[["new_index", "make", "model", "make_D1", "model_D1"]].tail(10))

,new_index,make,model,make_D1,model_D1
0,0.0,HONDA,CR-V,Honda,CR-V
1,1.0,HONDA,CIVIC,Honda,Civic
2,2.0,LEXUS,RX,Lexus,RX 450h
3,3.0,HONDA,CR-V,Honda,CR-V
4,4.0,HYUNDAI,SANTA FE,Hyundai,Santa Fe
5,5.0,GMC,SIERRA,GMC,Sierra 1500
6,6.0,NISSAN,MAXIMA,Nissan,Maxima
7,NaN,FREIGHTLINER,M2 SERIES,NaN,NaN
8,8.0,JEEP,WRANGLER,Jeep,Wrangler Unlimited
9,9.0,RAM,RAM,Ram,Ram Pickup 1500 Classic


,new_index,make,model,make_D1,model_D1
650587,650587.0,BMW,X4,BMW,X4
650588,650588.0,SUBARU,CROSSTREK,Subaru,Crosstrek
650589,650589.0,HYUNDAI,PALISADE,Hyundai,Palisade
650590,650590.0,VOLKSWAGEN,ATLAS,Volkswagen,Atlas
650591,650591.0,GMC,SIERRA,GMC,Sierra 1500
650592,650592.0,CHEVROLET,TAHOE,Chevrolet,Tahoe
650593,650593.0,VOLVO,V90,Volvo,V90
650594,650594.0,SUBARU,CROSSTREK,Subaru,Crosstrek
650595,650595.0,HONDA,CR-V,Honda,CR-V
650596,650596.0,RAM,RAM,Ram,Ram Pickup 1500


In [73]:
# Save as a CSV
rlp_d1_merge.to_csv(rlpolk_data_path / "rlp_d1_merge_020724.csv")

In [75]:
# Load in the DEF_ENGINE data 
def_engine = pd.read_csv(data_one_vins_path / "DEF_ENGINE.csv")

In [77]:
# Merge the DEF_ENGINE data with the rlp_d1_merge data on the engine_name column in the former
rlp_d1_merge_engine = rlp_d1_merge.merge(def_engine, left_on = "engine_id_D1", right_on = "engine_id", how = "left")

In [80]:
# Calculate how many records were dropped in that merge operation
print(f"Records dropped in merge with DEF_ENGINE: {len(rlp_d1_merge) - len(rlp_d1_merge_engine)}")

# How many did not match?
print(f"Records with no engine match: {len(rlp_d1_merge_engine[rlp_d1_merge_engine.engine_id.isna()])}")

Records dropped in merge with DEF_ENGINE: 0
Records with no engine match: 0


In [3]:
# Save again with the same location and filename
# rlp_d1_merge_engine.to_csv(rlpolk_data_path / "rlp_d1_merge_engine_020724.csv")
rlp_d1_merge_engine = pd.read_csv(rlpolk_data_path / "rlp_d1_merge_engine_020724.csv", index_col = [0])



In [27]:
[x for x in rlp_d1_merge_engine.columns if "wheelbase" in x]

['wheelbase_D1']

In [31]:
# Get value counts for wheelbase_D1 including NA
print(rlp_d1_merge_engine['wheelbase_D1'].value_counts(dropna = False).sum())

# Print the length of the df
print(len(rlp_d1_merge_engine))

# OK so we have wheelbase values for all the entries... 

602222
602222


Our goal is to ensure we have the same columns as in the original file.

`logit_formulation = pyblp.Formulation('0 + prices + dollar_per_mile + electric + phev + hybrid + diesel + log_hp_weight + wheelbase + doors + range_elec + chargers_per_tsm +  C(make) + C(drivetype) + C(bodytype)')`

# New RLP Data

In [ ]:
new_rlp = pd.read_table(rlpolk_data_path / "new" / "US_Yale_University_OP0001635118_NV_MA_CT_By_Zip_202310.txt", sep="|", header= 0)

In [ ]:
# make column names lower case
new_rlp.columns = new_rlp.columns.str.lower()

# split REPORT_YEAR_MONTH into two separate columns
new_rlp["year"] = new_rlp["report_year_month"].astype(str).str[0:4].astype(int)
new_rlp["month"] = new_rlp["report_year_month"].astype(str).str[4:6].astype(int)

# Extract only entries for Connecticut
new_rlp_ct = new_rlp[new_rlp["state_abbrv"] == "CT"]


,report_year_month,make,model,model_year,transaction_price,zip_code,state_abbrv,veh_count,year,month
0,202102,RAM,PROMASTER,2021,NaN,6371,CT,2,2021,2
1,202201,FORD,ESCAPE,2022,NaN,6410,CT,1,2022,1
2,201909,FORD,TRANSIT CONNECT VAN,2019,25100.0,6032,CT,1,2019,9
3,202304,JEEP,GRAND CHEROKEE,2023,NaN,6032,CT,1,2023,4
5,202305,CHEVROLET,SILVERADO,2023,48908.0,6023,CT,1,2023,5
...,...,...,...,...,...,...,...,...,...,...
1806321,201903,SUBARU,IMPREZA,2019,27374.0,6902,CT,1,2019,3
1806322,201902,MAZDA,CX-5,2019,29393.0,6902,CT,1,2019,2
1806324,202310,SUBARU,FORESTER,2023,30951.0,6066,CT,1,2023,10
1806326,201912,SUBARU,IMPREZA,2019,23967.0,6457,CT,1,2019,12


In [ ]:
# Show which years are available
new_rlp_ct.columns

# What are the lengths of t

Index(['report_year_month', 'make', 'model', 'model_year', 'transaction_price',
       'zip_code', 'state_abbrv', 'veh_count', 'year', 'month'],
      dtype='object')

In [ ]:
# Show 10 entries where the state is CT
new_rlp.loc[:, "state_abbrv"].unique()

array(['VA', 'OK', 'NC', 'TX', 'FL', 'NY', 'NJ', 'OH', 'CA', 'MA', 'IL',
       'MI', 'PA', 'GA', 'AZ'], dtype=object)

# View EV Characteristics Downloaded from NHTSA to check completeness

In [ ]:
ev_chars = pd.read_csv(rlpolk_data_path / "rlp_ev_characteristics_012624.csv", index_col = [0])

FileNotFoundError: [Errno 2] No such file or directory: '/Users/rrn22/Documents/tobin_working_data/rlpolk_data/rlp_ev_characteristics_012624.csv'

In [ ]:
ev_chars.columns

In [ ]:
evs_only_tomerge = evs_only.loc[:, ~evs_only.columns.isin(ev_chars.columns[0:len(ev_chars.columns)-1])]

In [ ]:
evs_only_chars = evs_only_tomerge.merge(ev_chars, how = 'left', on='vin_corrected')

In [ ]:
ev_notna_summary = evs_only_chars.loc[:, evs_only_chars.columns.isin(ev_chars.columns)]
ev_notna_summary = pd.DataFrame(ev_notna_summary.notna().sum()).reset_index().rename({0: "Count"}, axis = 1)
ev_notna_summary["Percentage"] = ev_notna_summary["Count"]/len(evs_only)

In [ ]:
ev_notna_summary.sort_values("Count", ascending = False).head(70)

,index,Count,Percentage
0,vin_corrected,29592,1.000000
122,Custom Motorcycle Type,27885,0.942315
124,Motorcycle Chassis Type,27885,0.942315
123,Motorcycle Suspension Type,27885,0.942315
9,Manufacturer Name,27885,0.942315
11,Model Year,27885,0.942315
15,Vehicle Type,27885,0.942315
8,Make,27885,0.942315
6,Vehicle Descriptor,27885,0.942315
40,Trailer Body Type,27885,0.942315


In [ ]:
vars_of_interest = ["vin_corrected", "Make", "Vehicle Type", "Fuel Type - Primary",
                    "Gross Vehicle Weight From", "Doors", "Number of Seats", "Curb Weight (pounds)",
                    "Transmission Style", "Wheel Base (inches) From", "Engine Brake (hp) From", "Displacement (CI)"]

ev_notna_keyvars = ev_notna_summary.loc[ev_notna_summary["index"].isin(vars_of_interest), :].reset_index(drop = True).sort_values("Percentage", ascending = False)

In [ ]:
ev_notna_keyvars

,index,Count,Percentage
0,vin_corrected,29592,1.000000
1,Make,27885,0.942315
2,Vehicle Type,27885,0.942315
9,Fuel Type - Primary,27800,0.939443
3,Doors,26415,0.892640
6,Number of Seats,21723,0.734084
7,Transmission Style,20834,0.704042
5,Wheel Base (inches) From,13680,0.462287
10,Engine Brake (hp) From,10550,0.356515
8,Displacement (CI),8882,0.300149


In [ ]:
print(ev_notna_keyvars.to_latex(index=False))

\begin{tabular}{lrr}
\toprule
index & Count & Percentage \\
\midrule
vin_corrected & 29592 & 1.000000 \\
Make & 27885 & 0.942315 \\
Vehicle Type & 27885 & 0.942315 \\
Fuel Type - Primary & 27800 & 0.939443 \\
Doors & 26415 & 0.892640 \\
Number of Seats & 21723 & 0.734084 \\
Transmission Style & 20834 & 0.704042 \\
Wheel Base (inches) From & 13680 & 0.462287 \\
Engine Brake (hp) From & 10550 & 0.356515 \\
Displacement (CI) & 8882 & 0.300149 \\
Curb Weight (pounds) & 366 & 0.012368 \\
\bottomrule
\end{tabular}

